<a href="https://www.kaggle.com/code/fawabc/formula-one-image-classification?scriptVersionId=144083177" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# installation
!pip install tensorflow tensorflow-gpu==2.8.0 opencv-python matplotlib numpy==1.23.0

In [ ]:
# import necessary libraries and modules
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy
from tensorflow.keras.models import load_model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# define the data directory path
data_dir = '/kaggle/input/formula-one-cars-images'

In [ ]:
# build image dataset pipeline
data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/formula-one-racecar-images/Formula One Cars')

In [ ]:
# convert dataset to numpy iterator for easy access of data as a pipeline
data_iterator = data.as_numpy_iterator()

In [ ]:
# accessing data through extractaction of a single batch from the dataset
batch = data_iterator.next()

In [ ]:
# get the class names from the dataset
class_names = data.class_names

In [ ]:
# create a mapping from label index to class names
label_to_class = {i: class_name for i, class_name in enumerate(class_names)}
label_to_class

In [ ]:
# convert batch labels to class names
batch_class_names = [label_to_class[label] for label in label_to_class]
batch_class_names

In [ ]:
# scale the image data to range [0, 1]
scaled_data = data.map(lambda x,y: (x/255, y))

In [ ]:
# partition train
train_size = int(len(scaled_data)*0.7)

In [ ]:
# partition test
test_size = int(len(scaled_data)*0.1)+1

In [ ]:
# partition validation
val_size = int(len(scaled_data)*0.2)+1

In [ ]:
# split data
train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)
test = scaled_data.skip(train_size+val_size).take(test_size)

In [ ]:
# initialize Sequential model
model = Sequential()

In [ ]:
# add convolutional and pooling layers
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())


# add dense layers for classification
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [ ]:
# compile model
model.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# get summary of model
model.summary()

In [ ]:
# set the log directory to view history 
logdir =' logs'

In [ ]:
# set history
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# train model
model_training = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
# visualise loss
fig = plt.figure()
plt.plot(model_training.history['loss'], color='teal', label='loss')
plt.plot(model_training.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# visualise accuracy
fig = plt.figure()
plt.plot(model_training.history['accuracy'], color='teal', label='accuracy')
plt.plot(model_training.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# set accuracy
accuracy = SparseCategoricalAccuracy()

In [ ]:
# get accuracy of model
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    accuracy.update_state(y, yhat)

In [ ]:
print('Accuracy:', accuracy.result().numpy())

In [ ]:
# test model on unforseen image - McLaren F1 Racecar
img = cv2.imread('/kaggle/input/unseen-formula-one-racecar-images/lando-norris-mclaren-mcl35m-1.jpg')
plt.imshow(img)
plt.show()

In [ ]:
# convert rgb
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# plot fixed rgb image
plt.imshow(img_rgb)
plt.title("Original Image")
plt.show()

In [ ]:
# normalise for model
input_shape = (256, 256)
resized_img = cv2.resize(img_rgb, input_shape)
resized_img_normalized = resized_img / 255.0
input_data = np.expand_dims(resized_img_normalized, axis=0)

In [ ]:
# predict
yhat = model.predict(input_data)
yhat

In [ ]:
# map out the racecars 
sorted_indices = np.argsort(yhat[0])[::-1]

In [ ]:
# print predictions
for i, index in enumerate(sorted_indices):
    prob = yhat[0][index]
    class_name = label_to_class[index]
    print(f'{i+1}th Prediction: {class_name} with {prob*100:.2f}% confidence.')

In [ ]:
# save model
model.save(os.path.join('models','f1-racecar-image-classifier.h5'))